# Numo::NArray : Universal Functions

`Numo::NArray`での計算は、書き方によって遅くなったり速くなったりします。高速化の鍵となるのは、 *Universal Function* です。

例えば、`NArray`の要素の逆数を求めるという問題を考えてみましょう。

## 遅い書き方：ループ

素直にループを使って書くと、以下のようになるでしょう：

In [1]:
require 'numo/narray'

def compute_reciprocals(values)
  output = Numo::DFloat.zeros(values.length)
  values.length.times do |i|
    output[i] = 1.0 / values[i]
  end
  return output
end

values = Numo::DFloat.new(5).rand(1,10)
compute_reciprocals(values)

Numo::DFloat#shape=[5]
[0.64276, 0.229484, 0.122649, 0.355951, 0.489149]

しかし、このアプローチでは、配列が大きいときにとても時間がかかります：

In [2]:
require 'benchmark'

big_array = Numo::DFloat.new(10000000).rand(1,100)
Benchmark.bm do |r|
  r.report 'rec' do
    compute_reciprocals(big_array)
  end
end
nil

       user     system      total        real
rec  1.950000   0.030000   1.980000 (  1.979000)


上の例では、$10^8$個の要素の逆数を求めるのに、2秒程度かかっています。

## 速い書き方：Universal Functions

`NArray`は、幾つかの演算について、Universal Functionを提供しています。Universal Functionを使うことで、先ほどの演算は、次のように書けます：

In [3]:
p compute_reciprocals(values)
p (1.0 / values)
nil

Numo::DFloat#shape=[5]
[0.64276, 0.229484, 0.122649, 0.355951, 0.489149]
Numo::DFloat#shape=[5]
[0.64276, 0.229484, 0.122649, 0.355951, 0.489149]


先程と同様に、計算時間を計測してみましょう：

In [4]:
Benchmark.bm do |r|
  r.report 'rec' do
    1.0 / big_array
  end
end
nil

       user     system      total        real
rec  0.060000   0.030000   0.090000 (  0.140044)


上のように、$10^8$個の要素の逆数を求めるのにかかる時間が、70ms程度まで短縮されていることがわかります。

ufuncは、2つの`NArray`に対して適用することもできます：

In [5]:
Numo::DFloat.new(5).seq(1) / Numo::DFloat.new(5).seq(2)

Numo::DFloat#shape=[5]
[0.5, 0.666667, 0.75, 0.8, 0.833333]

多次元の`NArray`に対して使うこともできます：

In [6]:
2 ** Numo::Int32.new(3,3).seq

Numo::Int32#shape=[3,3]
[[1, 2, 4], 
 [8, 16, 32], 
 [64, 128, 256]]

## ufuncの使い方

`NArray`のufuncとして、四則演算(`+-*/`)の他に、negation(`-`)や、累乗(`**`)、剰余(`%`)があります：

In [16]:
x = Numo::DFloat.new(4).seq(0)
puts("x      = #{x.to_a}")
puts("x + 5  = #{(x + 5).to_a}")
puts("x - 5  = #{(x - 5).to_a}")
puts("x * 2  = #{(x * 2).to_a}")
puts("x / 2  = #{(x / 2).to_a}")
puts("-x     = #{(-x).to_a}")
puts("x ** 2 = #{(x ** 2).to_a}")
puts("x % 2  = #{(x % 2).to_a}")

x      = [0.0, 1.0, 2.0, 3.0]
x + 5  = [5.0, 6.0, 7.0, 8.0]
x - 5  = [-5.0, -4.0, -3.0, -2.0]
x * 2  = [0.0, 2.0, 4.0, 6.0]
x / 2  = [0.0, 0.5, 1.0, 1.5]
-x     = [-0.0, -1.0, -2.0, -3.0]
x ** 2 = [0.0, 1.0, 4.0, 9.0]
x % 2  = [0.0, 1.0, 0.0, 1.0]


また、これらは、組み合わせて使うことができ、その場合は通常の演算子の優先順序が適用されます：

In [17]:
-(0.5*x + 1) ** 2

Numo::DFloat#shape=[4]
[-1, -2.25, -4, -6.25]

### 絶対値

`abs`メソッドによって、`NArray`の各要素の絶対値を求めることができます：

In [19]:
x = Numo::DFloat.new(5).store [-2, -1, 0, 1, 2]
x.abs

Numo::DFloat#shape=[5]
[2, 1, 0, 1, 2]

`NArray`は複素数を扱うことができますが、この場合`abs`は*複素数の絶対値*を計算します：

In [21]:
x = Numo::DComplex.new(4).store [Complex(3, -4), Complex(4, -3), Complex(2, 0), Complex(0, 1)]
x.abs

Numo::DFloat#shape=[4]
[5, 5, 2, 1]

### 三角関数

`NArray`には多くのufuncが定義されています。三角関数はその一つです：

In [23]:
theta = Numo::DFloat.new(3).seq(0, Math::PI/2)

puts("theta　　　= #{theta.to_a}")
puts("sin(theta) = #{Numo::NMath.sin(theta).to_a}")
puts("cos(theta) = #{Numo::NMath.cos(theta).to_a}")
puts("tan(theta) = #{Numo::NMath.tan(theta).to_a}")

theta　　　= [0.0, 1.5707963267948966, 3.141592653589793]
sin(theta) = [0.0, 1.0, 1.2246467991473532e-16]
cos(theta) = [1.0, 6.123233995736766e-17, -1.0]
tan(theta) = [0.0, 1.633123935319537e+16, -1.2246467991473532e-16]


逆三角関数も定義されています：

In [25]:
x = Numo::DFloat.new(3).store [-1, 0, 1]

puts("x         = #{x.to_a}")
puts("arcsin(x) = #{Numo::NMath.asin(x).to_a}")
puts("arccos(x) = #{Numo::NMath.acos(x).to_a}")
puts("arctan(x) = #{Numo::NMath.atan(x).to_a}")

x         = [-1.0, 0.0, 1.0]
arcsin(x) = [-1.5707963267948966, 0.0, 1.5707963267948966]
arccos(x) = [3.141592653589793, 1.5707963267948966, 0.0]
arctan(x) = [-0.7853981633974483, 0.0, 0.7853981633974483]


### 指数と対数

指数関数も定義されています：

In [27]:
x = Numo::DFloat.new(3).store [1,2,3]

puts("x   = #{x.to_a}")
puts("e^x = #{Numo::NMath.exp(x).to_a}")
puts("2^x = #{Numo::NMath.exp2(x).to_a}")
#puts("3^x = #{Numo::NMath.power(3, x).to_a}")

x   = [1.0, 2.0, 3.0]
e^x = [2.718281828459045, 7.38905609893065, 20.085536923187668]
2^x = [2.0, 4.0, 8.0]


対数関数も定義されています：

In [14]:
x = Numo::DFloat.new(4).store [1,2,4,10]

puts("ln(x)    = #{Numo::NMath.log(x).to_a}")
puts("log2(x)  = #{Numo::NMath.log2(x).to_a}")
puts("log10(x) = #{Numo::NMath.log10(x).to_a}")

ln(x)    = [0.0, 0.6931471805599453, 1.3862943611198906, 2.302585092994046]
log2(x)  = [0.0, 1.0, 2.0, 3.321928094887362]
log10(x) = [0.0, 0.3010299956639812, 0.6020599913279624, 1.0]


また、精度を保つための特殊な関数も定義されています：

In [29]:
# x = [0, 0.001, 0.01, 0.1]
# puts("exp(x) - 1 = #{Numo::NMath.expm1(x)}")
# puts("log(1 + x) = #{Numo::NMath.log1p(x)}")

### その他のufuncs

In [30]:
# numpyだと、scipyにたくさんufuncが定義されている
# ex:) gamma, gammaln, beta, erf, erfc, erfinv ...
# https://docs.scipy.org/doc/scipy/reference/special.html
# numo/narrayにそういう外部ライブラリみたいなのはない

## 高度なufuncの機能

### 出力先の指定

In [33]:
# x = Numo::DFloat.new(5).seq
# y = Numo::DFloat.new(5)
# y = x * 10
# ↑みたいなことが、numpyだと無駄なオブジェクトを生成すること無くできる.
# NArrayだと、一度(x * 10)を計算した結果のオブジェクトが生成されて、
# それにyという変数の参照先が向けられ、もともとyが指していたオブジェクトは廃棄される（おおまかには）。
# numpyだと、yが参照しているメモリに、直接 (x * 10)の結果が書き込まれるようにできる（たぶん）。

### aggregation

`NArray`には、幾つかの集約(aggregation)関数が定義されています（訳注 原文では、例えば`x.sum`は`x.add.reduce`のような形で記載されている）：

In [39]:
x = Numo::DFloat.new(5).seq(1)
x.sum

15.0

In [40]:
x.prod

120.0

In [41]:
x.cumsum

Numo::DFloat#shape=[5]
[1, 3, 6, 10, 15]

In [42]:
x.cumprod

Numo::DFloat#shape=[5]
[1, 2, 6, 24, 120]

### 直積

（以下のようにすれば、直積(outer products)の計算をすることができるが、`outer`メソッドのようなメソッドがあるわけではない）

In [32]:
x = Numo::DFloat.new(1, 6).seq(1)
x.transpose * x

Numo::DFloat#shape=[6,6]
[[1, 2, 3, 4, 5, 6], 
 [2, 4, 6, 8, 10, 12], 
 [3, 6, 9, 12, 15, 18], 
 [4, 8, 12, 16, 20, 24], 
 [5, 10, 15, 20, 25, 30], 
 [6, 12, 18, 24, 30, 36]]